# Load libraries 

In [1]:
# ! pip install --user open3d # run if open3D is not installed, http://www.open3d.org/ (Zhou et al., 2018)
# ! pip install --user scikit-spatial
import os
import numpy as np
import pandas as pd
from utils_final import whole_pipeline

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Create result dataframe 

In [2]:
def pad(data, max_slices, pad_value=np.NAN):
    """
    Create padding for dataframe where there is no tree slice at that height
    
    Parameters
    ----------------
    data: 
        values of interest
    max_slices:
        maximum number of slices that make up a plot or individual tree
    pad_value:
        NAN if a tree doesn't reach the max height 
    
    Returns
    ----------------
    The correct amount of null values
    """    
    return np.pad(data, (0,max_slices-len(data)), 'constant', constant_values=pad_value)


def generate_df(file_name_list, value_list):
    """
    Creates result dataframe with null values and names based on the file name 
    
    Parameters
    ----------------
    file_name_list: 
        point cloud files (.pcd)
    value_list:
        LAI, leaf area density, or leaf angle values
        
    Returns
    ----------------
    Concatenated results dataframe of the appropriate species grouping
    """
    max_slices = np.max([len(x) for x in value_list])
    
    value_array_padded = np.array([pad(x,max_slices) for x in value_list])
    values_df = pd.DataFrame(value_array_padded,columns=[f"slice_{x+1}" for x in range(value_array_padded.shape[1])])    
    
    file_name_df = pd.DataFrame(
        {
            "plot_name":files_list,
        }
    )
    file_name_df = file_name_df["plot_name"].str.split("_",n=3,expand=True)
    file_name_df = file_name_df.drop(3,axis=1)

    file_name_df.loc[file_name_df[2] == "leaves",2] = "all"
    file_name_df.columns = ["plot","year","species"]
    
    return pd.concat([file_name_df,values_df],axis=1)

# Specify input & output folders

In [3]:
input_folder =  "C:\\Users\\ANGELASEIBERT\\Desktop\\Python_Workspace\\leaf_pcd_files\\testing\\" # where the leaf .pcd files are stored 
file_extension = ".pcd"
output_folder =  "C:\\Users\\ANGELASEIBERT\\Desktop\\Python_Workspace\\leaf_pcd_files\\testing\\output\\" # where the output csv should go

# Import the full workflow from utils

In [4]:
from utils_final import whole_pipeline # save utils file and run this line to apply any utils updates

# Run the full workflow 

In [6]:
files_list = []
LAI_list = []
LAD_list = []
allangles_list = []
LA_correction_factors_list = []

for file_1 in os.listdir(input_folder):
    if file_1.endswith(file_extension):
        files_list.append(file_1.split('.')[0])
        
        print("-"*10, file_1)
        number_words = len(file_1.split("_"))

        if number_words == 3:
            print("WHOLE PLOT")
            whole_path_1 = input_folder+file_1
            LAI,LAD,allangles,LA_correction_factors = whole_pipeline(whole_path_1,whole_path_1,visualize=False)
        elif number_words == 4:
            print("NOT WHOLE PLOT")
            # infer name of the file with the whole plot
            file_1_split = file_1.split("_")
            file_1_split.pop(2)
            file_2 = "_".join(file_1_split)

            whole_path_1 = input_folder+file_1
            whole_path_2 = input_folder+file_2

            LAI,LAD,allangles,LA_correction_factors = whole_pipeline(whole_path_1,whole_path_2,visualize=False) 

        LAI_list.append([LAI])
        LAD_list.append(LAD)
        allangles_list.append(allangles)
        LA_correction_factors_list.append(LA_correction_factors)
        
# storage in the final dataframe
LAI_df = generate_df(files_list,LAI_list)
LAI_df = LAI_df.rename(columns={'slice_1':'LAI'})

LAD_df = generate_df(files_list,LAD_list)
allangles_df = generate_df(files_list,allangles_list)
LA_correction_factors_df = generate_df(files_list,LA_correction_factors_list)


if not os.path.isdir(output_folder):
    os.mkdir(output_folder)
    
LAI_df.to_csv(output_folder+"LAI_out.csv",index=False)
LAD_df.to_csv(output_folder+"LAD_out.csv",index=False)
allangles_df.to_csv(output_folder+"leaf_angles_out.csv",index=False)
LA_correction_factors_df.to_csv(output_folder+"cf_out.csv",index=False)

---------- plot4_2015_larch_leaves.pcd
NOT WHOLE PLOT
how many half meter slices: 13.0
Slice Correction Factors: [0.0904020152610505, 0.20622116376906577, 0.28187204752856393, 0.3428476143895694, 0.36321084243959045, 0.3693582386373871, 0.3903613051058343, 0.46918265212630966, 0.42025898898486974, 0.40970635510595627, 0.48059263412381703, 0.44285345015145866, 0.4608425030414553]
CF Standard Deviations: [0.11009686104828306, 0.08915333550937744, 0.04671351593737734, 0.011845844703880459, 0.026560023549997744, 0.03772239886330231, 0.06957297841762224, 0.11751606974773897, 0.13135789512708151, 0.11976399395167393, 0.21188898136143447, 0.13797865174680218, 0.18455308615236193]
how many half meter slices for the second file: 13.0
The LAD is: [0.006101076219324704, 0.008787317340418585, 0.008578837064875935, 0.008225090329644469, 0.007153409109666555, 0.010626465113938843, 0.008230036029864732, 0.013789033993522262, 0.009638449581366322, 0.008993731059725596, 0.01136400971378208, 0.011585694